In [1]:
from notebook_environment import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

## CCLE data

In [2]:
# Download from https://portals.broadinstitute.org/ccle/data:
#     'mutation__gene_x_ccle_cellline.gct'
#     'rpkm__gene_x_ccle_cellline.gct'
#     'rppa__protein_x_ccle_cellline.gct'
#     'annotation__feature_x_ccle_cellline.gct' or 'CCLE_sample_info_file_2012-10-18.txt'
#         df = pd.read_table('../data/CCLE_sample_info_file_2012-10-18.txt', index_col=0)
#         gct.write_gct(df.T, '../data/annotation__feature_x_ccle_cellline.gct')

## Achilles data

In [3]:
# Download https://portals.broadinstitute.org/achilles/datasets/15/download/ExpandedGeneZSolsCleaned.csv

df = pd.read_csv('../data/ExpandedGeneZSolsCleaned.csv', index_col=0)
gct.write_gct(df, '../data/achilles__gene_x_ccle_cellline.gct')

## CTD^2 data

In [5]:
# Download ftp://caftpd.nci.nih.gov/pub/OCG-DCC/CTD2/Broad/CTRPv2.2_2015_pub_CancerDisc_5_1210/CTRPv2.2_2015_pub_CancerDisc_5_1210.zip

# Unzip
file.unzip('../data/CTRPv2.2_2015_pub_CancerDisc_5_1210.zip')

# Read compound data
auc = pd.read_table('../data/v22.data.auc_sensitivities.txt')
print(auc.shape)

cpd = pd.read_table('../data/v22.meta.per_compound.txt', index_col=0)
print(cpd.shape)

ccl = pd.read_table('../data/v22.meta.per_cell_line.txt', index_col=0)
print(ccl.shape)

# Make dict for faster ID-to-name look up
cpd_d = cpd['cpd_name'].to_dict()
ccl_d = ccl['ccl_name'].to_dict()

# Make empty compound-x-cellline matrix
compound_x_cellline = pd.DataFrame(
    index=sorted(set(cpd['cpd_name'])), columns=sorted(set(ccl['ccl_name'])))
print(compound_x_cellline.shape)

# Populate compound-x-cellline matrix
for i, (i_cpd, i_ccl, a) in auc.iterrows():

    # Get compound name
    cpd_n = cpd_d[i_cpd]

    # Get cellline name
    ccl_n = ccl_d[i_ccl]

    # Get current AUC
    a_ = compound_x_cellline.loc[cpd_n, ccl_n]

    # If the current AUC is not set, set with this AUC
    if pd.isnull(a_):
        compound_x_cellline.loc[cpd_n, ccl_n] = a

    # If this AUC is smaller than the current AUC, set with this AUC
    elif a < a_:

        print('Updating AUC of compound {} on cellline {}: {:.3f} ==> {:.3f}'.
              format(cpd_n, ccl_n, a_, a))

        compound_x_cellline.loc[cpd_n, ccl_n] = a

# Update cellline names to match CCLE cellline names
columns = list(compound_x_cellline.columns)

# Read CCLE cellline annotations
a = pd.read_table('../data/CCLE_sample_info_file_2012-10-18.txt', index_col=0)

# Get CCLE cellline names
for i, ccl_n in enumerate(compound_x_cellline.columns):

    matches = []

    for ccle_n in a.index:
        if ccl_n.lower() == ccle_n.lower().split('_')[0]:
            matches.append(ccle_n)

    if 0 == len(matches):
        print('0 match: {}; matching substring ...'.format(ccl_n))

        for ccle_n in an.index:

            if ccl_n.lower() in ccle_n.lower():

                print('\t{} ==> {}.'.format(ccl_n, ccle_n))
                matches.append(ccle_n)

    if 1 == len(matches):

        print('{} ==> {}.'.format(ccl_n, matches[0]))
        columns[i] = matches[0]

    else:
        print('1 < matches: {} ==> {}'.format(ccl_n, matches))

# Update with CCLE cellline names
compound_x_cellline.columns = columns

# Write .gct file
gct.write_gct(compound_x_cellline,
              '../data/ctd2__compound_x_ccle_cellline.gct')

compound_x_cellline

In [ ]:
# Check if all data files exist

for fn in [
        'kras_isogenic_and_imortalized_celllines.gct',
        'mutation__gene_x_ccle_cellline.gct',
        'rpkm__gene_x_ccle_cellline.gct',
        'gene_set__gene_set_x_ccle_cellline.gct',
        'regulator__gene_set_x_ccle_cellline.gct',
        'rppa__protein_x_ccle_cellline.gct',
        'ctd2__compound_x_ccle_cellline.gct',
        'annotation__feature_x_ccle_cellline.gct',
]:
    assert fn in os.listdir('../data/'), 'Missing {}!'.format(fn)